In [1]:
import pathlib
import requests
from black.trans import defaultdict

year = 2024
day = 15

url = f"https://adventofcode.com/{year}/day/{day}/input"

with open("cookie.txt", "r") as file:
    session = file.read()

headers = {"cookie": f"session={session}"}

pathlib.Path("inputs").mkdir(parents=True, exist_ok=True)

with open(f"inputs/day_{str(day).zfill(2)}_input.txt", "wb") as file:
    file.write(requests.get(url, headers=headers).content)

In [39]:
from copy import deepcopy

lines = [x for x in open(f"inputs/day_{str(day).zfill(2)}_input.txt", "r")]
aoc = "".join(lines)
G = [[x for x in line.strip()] for line in lines]
R = len(G)
C = len(G[0])

print(lines[:5])
print(R, C)
start_lines = deepcopy(lines)

['##################################################\n', '#....OO..O#...O...##....##.....O.....OO..O....O..#\n', '##.....O...O.......O.......O.O.OOO.O.O...OO.#..#O#\n', '##..#.O.O....#..O.OOO...#.........O..........O...#\n', '#O..#O....O.OOO.O............O....O......#O.O....#\n']
71 50


In [123]:
dirs = {"^": (-1, 0), ">": (0, 1), "v": (1, 0), "<": (0, -1)}
debug = False

sep = "\n"
lines = deepcopy(start_lines)

grid = lines[: lines.index(sep)]

moves = lines[lines.index(sep) :]
G = [[x for x in line.strip()] for line in grid]
R = len(G)
C = len(G[0])

moves = [move for sublist in moves for move in sublist.strip()]

walls = []
boxes = []
robot = None

for r, row in enumerate(G):
    for c, cell in enumerate(row):
        if cell == "#":
            walls.append((r, c))
        if cell == "O":
            boxes.append((r, c))
        if cell == "@":
            robot = (r, c)


def try_to_move_box(box_index, dn):
    box = boxes[box_index]

    nr = box[0] + dn[0]
    nc = box[1] + dn[1]

    if (nr, nc) in walls:
        return False
    elif (nr, nc) in boxes:
        moved_next_box = try_to_move_box(boxes.index((nr, nc)), dn)

        if moved_next_box:
            boxes[box_index] = (nr, nc)
            return True
    else:
        boxes[box_index] = (nr, nc)
        return True


def print_grid():
    for r, row in enumerate(G):
        str_row = ""
        for c, cell in enumerate(row):
            if (r, c) in walls:
                str_row += "#"
            elif (r, c) in boxes:
                str_row += "O"
            elif (r, c) == robot:
                str_row += "@"
            else:
                str_row += "."
        print(str_row)
    print(*"\n")


if debug:
    print_grid()

for move in moves:
    dn = dirs[move]
    nr = robot[0] + dn[0]
    nc = robot[1] + dn[1]

    if (nr, nc) in walls:
        pass
    elif (nr, nc) in boxes:
        moved_box = try_to_move_box(boxes.index((nr, nc)), dn)
        if moved_box:
            robot = (nr, nc)
    else:
        robot = (nr, nc)

    if debug:
        print(move, dn)
        print_grid()

ans = 0
for box in boxes:
    ans += box[0] * 100 + box[1]

print(ans)

1414416


In [125]:
sep = "\n"
lines = deepcopy(start_lines)

grid = lines[: lines.index(sep)]

moves = lines[lines.index(sep) :]
G = [[x for x in line.strip()] for line in grid]
R = len(G)
C = len(G[0])

moves = [move for sublist in moves for move in sublist.strip()]


walls = []
boxes = {}
b = 0
robot = None

for r, row in enumerate(G):
    for c, cell in enumerate(row):
        if cell == "#":
            walls.append((r, 2 * c))
            walls.append((r, 2 * c + 1))
        if cell == "O":
            boxes[b] = {(r, 2 * c), (r, 2 * c + 1)}
            b += 1
        if cell == "@":
            robot = (r, c * 2)


def try_to_move_box(box_index, dn):

    list_to_move = set()

    for i, box in enumerate(boxes[box_index]):
        nr = box[0] + dn[0]
        nc = box[1] + dn[1]

        if (nr, nc) in walls:
            return []

        elif (nr, nc) in set.union(*boxes.values()):

            next_box_id = [k for k, v in boxes.items() if (nr, nc) in v]
            next_box_id = next_box_id[0]

            if next_box_id != box_index:
                rec_list_to_move = try_to_move_box(next_box_id, dn)

                if len(rec_list_to_move) > 0:
                    list_to_move.update(rec_list_to_move)
                else:
                    return []

        list_to_move.add(box_index)

    return list_to_move


def print_grid():
    for r, row in enumerate(range(R)):
        str_row = ""
        for c, cell in enumerate(range(2 * C)):
            if (r, c) in walls:
                str_row += "#"
            elif (r, c) in set.union(*boxes.values()):
                str_row += "O"
            elif (r, c) == robot:
                str_row += "@"
            else:
                str_row += "."
        print(str_row)
    print(*"\n")


if debug:
    print_grid()

for move in moves:
    dn = dirs[move]
    nr = robot[0] + dn[0]
    nc = robot[1] + dn[1]

    if (nr, nc) in walls:
        pass
    elif (nr, nc) in set.union(*boxes.values()):

        box_id = [k for k, v in boxes.items() if (nr, nc) in v][0]

        moved_box = try_to_move_box(box_id, dn)

        for box_id in moved_box:
            boxes[box_id] = {(r + dn[0], c + dn[1]) for (r, c) in boxes[box_id]}

        if moved_box:
            robot = (nr, nc)
    else:
        robot = (nr, nc)

    if debug:
        print_grid()
        print(boxes)

ans = 0
for box, box2 in boxes.values():
    if box[1] < box2[1]:
        ans += box[0] * 100 + box[1]
    else:
        ans += box2[0] * 100 + box2[1]

print(ans)

1386070
